In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy
using PDFIO
using Unzip
using Interpolations
using Plots
using TensorCast
#plotlyjs()
data_dir = "C:\\Cross-Section-Data\\EXFOR\\"

"C:\\Cross-Section-Data\\EXFOR\\"

#Design

    Looping check_line() through each line in the file detects where each dataset begins. 

    Reading the line above each output of check_line() gives the number of rows in each dataset.

    make_spacing_dict() gets the names of the columns and the instructions for how to read each dataset.

    read_dataset() reads each and constructs a dictionary from each dataset.
    
    read_exfor_file() runs all of the above in order to return a single DataFrame of all the data stored at a given file path.
     

In [2]:
function check_line(line, start)
    if length(line) > length(start) - 1
        return (line[1:length(start)] == start)
    end
    return false
end

function make_spacing_dict(line1, line2)
    spacing_ends = [collect(out)[1] for out in findall(">", line2)]
    spacing_starts = append!([1], [collect(out)[1] for out in findall("<", line2)])
    #Find where some of the spacings begin and end based on the arrows
    if length(spacing_ends) != length(spacing_starts)
        print("Error: improper formatting")
        return 
    end
    indices_with_gaps = [index for index in  1:length(spacing_ends)-1 if 
                    spacing_ends[index] != spacing_starts[index+1]-1]
    #Some of the spacings are instead denoted by the letter o instead of arrows
    missing_spacings = [spacing_ends[index]+1:spacing_starts[index+1]-1 for index in indices_with_gaps]
    spacings = append!(missing_spacings, 
    [spacing_starts[i]:spacing_ends[i] for i in 1:length(spacing_ends)])
    spacing_dict = Dict([])
    #Make a dictionary where the keys are name for each column and the values are the indices of the columns
    spacing_names = [spacing_dict[strip(line1[spacing], [' ', '#'])] = spacing for spacing in spacings]
    return spacing_dict
end

function read_datum(datum)
    #Reads a single datum from a line of data
    datum = strip(datum, [' '])
    out = tryparse(Float64, datum)
    if out == nothing
        return datum 
    end
    return out
end

function read_dataset(spacing_specifier, file_as_vector)
    index = spacing_specifier
    spacing_dict = make_spacing_dict(file_as_vector[index], file_as_vector[index + 1])
    lines_of_data = tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
    data_dict = Dict([])
    filler = [data_dict[key] = [read_datum(file_as_vector[line_num][spacing_dict[key]])
            for line_num in index + 2: index + 1 + lines_of_data] for key in keys(spacing_dict)]
    return data_dict#, lines_of_data
end

read_dataset (generic function with 1 method)

It it faster to use 1 or loop for $n$ list comprehensions? 

In [3]:
function read_exfor_file(file_path)
    #Reads an EXFOR file and returns a dictionary of data
    file_as_vector = readlines(file_path)
    spacing_specifiers = [index for index in 1:length(file_as_vector) 
                        if check_line(file_as_vector[index], "# Prj")]
    if spacing_specifiers == []
        print("Error: no data found")
        return
    end
    #rows_in_combined_df = sum([tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
    #                for index in spacing_specifiers])
    #Each dataset should have the same column names
    spacing_dict = make_spacing_dict(file_as_vector[spacing_specifiers[1]], 
                    file_as_vector[spacing_specifiers[1] + 1])
    spacing_keys = [key for key in keys(spacing_dict)]
    #make an empty dataframe to fill up with the data
    #df = DataFrame([Vector{Union{Missing, Float64, String, SubString{String}}}(missing, rows_in_combined_df) 
    #                for _ in 1:length(spacing_keys)], spacing_keys)
    #Fill the dataframe with the data
    #current_row = 1
    #This for loop is probably the slowest part of the code. It alone takes over 50% of the runtime.  
    print("hi")
    df = reduce(vcat,[DataFrame(read_dataset(spacing_specifiers[i], file_as_vector)) 
            for i in 1:length(spacing_specifiers)])
    #==
    for spacing_specifier in spacing_specifiers
        data_dict, rows_in_dataset = read_dataset(spacing_specifier, file_as_vector)
        _ = [df[current_row:current_row+rows_in_dataset-1, key] = data_dict[key]
            for key in keys(data_dict)]
        current_row += rows_in_dataset
    end
    ==#
    return df
end

read_exfor_file (generic function with 1 method)

In [4]:
file_path = data_dir * "neutrons\\001_H_001.c4"
read_exfor_file(file_path)

hi

,Cos/LO,Data,ELV/HL,Energy,Entry,I78,M,MF,MT
,Any,Any,Any,Any,Any,SubStrin…,SubStrin…,Float64,Float64
1,,2.5863,,1.25e6,V0015,,,1.0,452.0
2,,2.587,,1.275e6,V0015,,,1.0,452.0
3,,2.5878,,1.3e6,V0015,,,1.0,452.0
4,,2.5886,,1.325e6,V0015,,,1.0,452.0
5,,2.5896,,1.35e6,V0015,,,1.0,452.0
6,,2.5906,,1.375e6,V0015,,,1.0,452.0
7,,2.5917,,1.4e6,V0015,,,1.0,452.0
8,,2.5929,,1.425e6,V0015,,,1.0,452.0
9,,2.5941,,1.45e6,V0015,,,1.0,452.0
